# Analyse der KantonsrätInnen: Das Ranking zum Ende der Legislatur
Anlässlich der Kantonsratswahlen 2019 analysieren wir das Abstimmungsverhalten, die Anwesenheit und die Aktivität der Parlamentarier. Analyseitraum (Legislatur 15/19 bis Ende 2018)
- Daten für Abstimmung: Von den Parlamentsdiensten zur Verfügung gestellt (Legislatur bis Ende 2018(Amtsjahr 2018/2019 keine Codierung für Art der Abstimmung))
- Daten für Voralgen gescraped mit diesem Script:
https://github.com/spatrice/kantonsrat_pi/blob/master/kantonsrat_scraper.ipynb
- Artikel im Tages-Anzeiger erschienen am 11.3 (ABO+): https://www.tagesanzeiger.ch/zuerich/stadt/Das-exklusive-TopTenRanking-der-Zuercher-Politik/story/26224484

In [ ]:
#my standard library import
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import requests
from bs4 import BeautifulSoup
import PyPDF2
import re
import time
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from cycler import cycler
plt.rcParams.update(
    {"figure.facecolor": "#ffffff",
              "axes.facecolor": "#ffffff",
              "axes.grid" : True,
              "axes.grid.axis" : "y",
              "grid.color"    : "ededed",
              "grid.linewidth": 0.8,
              "grid.alpha": 0.8,
              "axes.spines.left" : False,
              "axes.spines.right" : False,
              "axes.spines.top" : False,
              "axes.spines.bottom" : False,
              "axes.axisbelow": False,
              "ytick.major.size": 0,     
              "ytick.minor.size": 0,
              #"xtick.direction" : "none",
              "xtick.minor.size": 0,
              "xtick.major.size" : 0,
              "xtick.color"      : "#191919",
              "axes.edgecolor"    :"#191919",
              "axes.prop_cycle" : plt.cycler('color', ['#0c2c84', '#c7e9b4', 
                                                       '#225ea8','#1d91c0', '#41b6c4', 
                                                       '#7fcdbb', '#eaea8c']),
    'pdf.fonttype': 42,
    'ps.fonttype': 42
})
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500

import glob


In [ ]:
#importiere alle .csv Daten, welche die Parlamentsdienste angeliefert haben
filenames = glob.glob('daten/Voting_Kantonsrat*.csv')
print('Zahl der Abstimmungen:',len(filenames))
#columnames['s_nr', 'g_name', 'g_nr',]
# 4,5,6,7,16,17

### Achtung: Daten sind tricky

Mergen zu einem Datensatz ist mit Schwierigkeiten verbunden. Daten der Parlamentsdienste enthalten keinen Header. Weiteres Problem: Anzahl der Spalten variiert zwischen 19 und 23. Manchmal fehlen Geschäftsnummern, manchmal Spalten am Schluss, manchmal auch dazwischen.

Lösung für unsere Analyse: Wir brauchen nur die Namen, die Filenamen und das individuelle Abstimmungsverhalten, sowie für die Abwesenden den Abstimmungstyp. 

Deshalb Logik in Loop eingebaut: Script sucht das Geschlecht / Anrede (Herr/Frau), nimmt diese Spalte und die vier darauffolgenden. Danach schaut das Script, in welcher Spalte erstmals die Werte J, N, A oder E vorkommen. Das ist die Spalte mit dem individuellen Abstimmungsverhalten. Für die Abstimmungsart verwendet der Loop erneut das Geschlecht als Ankerpunkt. In allen Files ist die Abstimmungsart 3 Spalten neben dem Geschlecht.

In [ ]:
df = pd.DataFrame()
koko = [0] #für den Loop, damit er koko kennt.
cc = 0
for filename in filenames:
    dataframes = pd.read_csv(filename, sep=';', encoding='Latin1', header=-1, dtype=str)
    if len(dataframes)>180:
        print(len(dataframes))
        dataframes = dataframes.dropna()
        print(len(dataframes))
    count = -1 #startet bei minus -1 damit erster Wert 0 ist.
    cc +=1 #für der printoutput
    gender = 0
    result = []
    for col in dataframes.columns:
        count +=1
        if dataframes[col].iloc[1] in ['Herr', 'Frau']: #ist Herr oder Frau eine Ausprägung, schreib count in gender
            gender = count
            koko = [kolon for kolon in range(gender, gender+4)] #liste mit Werte für Columns erstellen
            votetype = gender-3 #um später Quorums-Abstimmungen rauszufiltern
        if col > koko[-1]: #weil ich weiss, dass das individuelle Abstimmungsverhalten nach der letzten koko Spalte kommt und darauf nur Zahlen folgen    
            if dataframes[col].str.contains('[JNEA]').values[0]:
                  result.append(count) #weil Loop durchläuft liste füllen, erstes Element später wählen
    koko.append(result[0])
    koko.insert(0,votetype)
    dataframes = dataframes.iloc[:,koko]
    dataframes.columns = ['votetype','gender', 'nachname', 'vorname','p_partei', 'p_result']
    votetype_def = dataframes['votetype'].iloc[0]
    dataframes['votetype_def'] = votetype_def
    dataframes['filename'] = filename
    print("Noch", str(len(filenames)-cc), "müssen geladen werden.")
    df = pd.concat([df, dataframes], sort=False)
data = df
df.head()

In [ ]:
#Daten etwas aufbretzeln
df['gender']=df['gender'].str.strip()
df['jahr'] = df['filename'].str.extract('daten/Voting_Kantonsrat_(\d\d\d\d)')
df['datum'] = df['filename'].str.extract('daten/Voting_Kantonsrat_(\d\d\d\d_\d\d_\d\d)')
df['datum'] = df['datum'].str.replace('_', '/')
df['datum'] = pd.to_datetime(df['datum'], format="%Y/%m/%d")
df['vorname'] = df['vorname'].str.strip()
df['nachname'] = df['nachname'].str.strip()
df['name'] = df['vorname'] + ' ' + df['nachname'] + ' ' + '('+ df['p_partei'] + ')'
df = df.sort_values('datum')
#Ergänze Daten mit Präsidium, wichtig für Zählung der Abwesenden
df['amtsjahr'] = 0
df['amtsjahr'][(df['datum']>='2015-05-18') & (df['datum']<'2016-05-08')] = 1
df['amtsjahr'][(df['datum']>='2016-05-08') & (df['datum']<'2017-05-08')] = 2
df['amtsjahr'][(df['datum']>='2017-05-08') & (df['datum']<'2018-05-07')] = 3
df['amtsjahr'][(df['datum']>='2018-05-07')] = 4
#Ergänze Daten mit Präsidium, wichtig für Zählung der Abwesenden
df['praesidium'] = 0
df['praesidium'][(df['datum']>='2015-05-18') & (df['datum']<'2016-05-08')] = 'Theresia Weber-Gachnang (SVP)'
df['praesidium'][(df['datum']>='2016-05-08') & (df['datum']<'2017-05-08')] = 'Rolf Steiner (SP)'
df['praesidium'][(df['datum']>='2017-05-08') & (df['datum']<'2018-05-07')] = 'Karin Egli (SVP)'
df['praesidium'][(df['datum']>='2018-05-07')] = 'Yvonne Bürgin (CVP)'
df.head()

In [ ]:
#Datensatz speichern
#df.to_csv('abstimmungsdaten_neu.csv',index=False)

### 1. Aanalyse: die Abwesenden
Achtung: amtsjahr 2018/2019 ist noch nicht codiert. muss deshalb ausgelassen werden.

In [ ]:
#Daten laden
df = pd.read_csv('abstimmungsdaten_neu.csv')
df.head()

In [ ]:
#Überprüfen, ob Codierung stimmt
#df[df['amtsjahr']==1].votetype_def.value_counts() #kein 0
df[df['amtsjahr']==2].votetype_def.value_counts() #einmal eine 0
#df[df['amtsjahr']==3].votetype_def.value_counts() #dreimal eine 0
#df[df['amtsjahr']==4].votetype_def.value_counts()

In [ ]:
#schaue problematische datensätze an, die 0 haben und nicht im noch nicht codierten amtsjahr sind
#2016/2017
nuller2 = list(set(df[(df['votetype_def']==0) & (df['amtsjahr']==2)].filename))
nullcheck2 = pd.read_csv(nuller2[0], sep=';', encoding='Latin1', header=-1, dtype=str)

In [ ]:
#wird als standard codiert mit 1
df['votetype_def'][df['filename']==nuller2[0]] = 1
df[df['filename']==nuller2[0]].head()

In [ ]:
#2017/2018
nuller3 = list(set(df[(df['votetype_def']==0) & (df['amtsjahr']==3)].filename))
nullcheck31 = pd.read_csv(nuller3[0], sep=';', encoding='Latin1', header=-1, dtype=str)
nullcheck32 = pd.read_csv(nuller3[1], sep=';', encoding='Latin1', header=-1, dtype=str)
nullcheck33 = pd.read_csv(nuller3[2], sep=';', encoding='Latin1', header=-1, dtype=str)
#nullcheck31
#nullcheck32
#nullcheck33

In [ ]:
#1 wird als quorum codiert mit 2 | PI vorläufige unterstützung | erste zeile korrekt erfasst dann nicht mehr
df['votetype_def'][df['filename']==nuller3[0]] = 2
df[df['filename']==nuller3[0]].head()
#2 wird als standard codiert mit 1 | abstimmung über Volksinitiative | plötzlich in spalte richtig
df['votetype_def'][df['filename']==nuller3[1]] = 1
df[df['filename']==nuller3[1]].head()
#3 wird als standard codiert mit 1 | Standard abstimmung gesetz | erste zeile falsch
df['votetype_def'][df['filename']==nuller3[1]] = 1
df[df['filename']==nuller3[1]].head()
#mache alle float
df['votetype_def'] = df['votetype_def'].astype(float)

In [ ]:
#subset für ersten drei amtsjahre und ohne quorums-abstimmungen
df18 = df[(df['amtsjahr']<=3) & (df['votetype_def']!=2)]
#df18 = df[(df['amtsjahr']<=3)]

len(set(df18.filename))

In [ ]:
#Gruppiere nach Name, Zähle, wie häufig Resultat == A | da PräsidentInnen nicht abstimmen, dort immer A, dieser rausrechnen
abwesende = df18[['name','p_result']][(df18['p_result']=='A') & (df18['name']!=df18['praesidium'])].groupby('name').count().sort_values('p_result',ascending=False)
abwesende= abwesende.reset_index()
abwesende.head(100)
#abwesende.to_csv('abwesende_bis17_18_ohnequorum.csv',index=False)


### 2. Aanalyse: die Abweichler
Als Abweichler gilt, wer gegen die Mehrheit seiner Fraktion gestimmt hat.

In [ ]:
#Wie hat die Partei abgestimmt?
partyvotes = df.groupby(['p_partei','filename'])['p_result'].value_counts().unstack().reset_index()
partyvotes.fillna(0, inplace=True)

In [ ]:
partyvotes['party_result'] = 0
partyvotes['party_result'][(partyvotes['J'] > partyvotes['N'])] = 'J'
partyvotes['party_result'][(partyvotes['J'] < partyvotes['N'])] = 'N'
partyvotes['party_result'][(partyvotes['J'] == partyvotes['N'])] = 'P'

#falls alle enthalten oder abwesend
partyvotes['party_result'][(partyvotes['J']==0) & (partyvotes['N'] == 0) & (partyvotes['A'] < partyvotes['E'])] = 'E'
partyvotes['party_result'][(partyvotes['J']==0) & (partyvotes['N'] == 0) & (partyvotes['A'] > partyvotes['E'])] = 'A'

In [ ]:
#Merge die beiden Datensätze nach Partei und Filename (Geschäft)
df = pd.merge(df, partyvotes, left_on=['p_partei', 'filename'], right_on=['p_partei', 'filename'], how='left').sort_values('filename')

In [ ]:
#Berechne Abweichler
#Definition Abweichler: Nicht mit der Fraktionsmehrheit gestimmt. Bei Patt, kein Abweichen möglich. Enthalten/Abwesend kein aktives Abweichen
df['abweichler'] = 0
df['abweichler'][(df['p_result']!='A') & (df['p_result']!='E') & (df['p_result']!=df['party_result']) & (df['party_result']!='P')] = 1

In [ ]:
abweichler = df[['name','abweichler']].groupby(['name']).sum().sort_values('abweichler', ascending=False)
abweichler = abweichler.reset_index()
abweichler.head(10)
#abweichler.to_csv('abweichler.csv',index=False)

### 3. Aanalyse: die Aktiven
Hier werden noch einige Unterscheidungen nach Newcomer und Alteingesessenen durchgeführt

In [ ]:
df = pd.read_csv('vorstoesse_parlamentariercheck.csv') #csv erstellt mit Scraper
df = df[df['year']>=2015]
notinlegis = df[(df['year']==2015) & (df['nr']<145)].index#erster Vorstoss der Legislatur 145/2015
df.drop(df.index[notinlegis], inplace=True)
df = df.dropna()
df.head()

In [ ]:
#im Datensatz nur Erstunterzeichner. Bei Gleichstand sollen Mitunterzeichnungen entscheiden.
def get_mitunterzeichner_1(row):
    anzahl_mitunterzeichner = len(row['beteiligte'].split('\n'))
    mitunterzeichner = row['beteiligte'].split('\n')
    if anzahl_mitunterzeichner < 3:
        mitunterzeichner.append('NaN'*(3-anzahl_mitunterzeichner))
    if len(mitunterzeichner) < 3:
        mitunterzeichner.append('NaN'*(3-(len(mitunterzeichner))))
    return mitunterzeichner[1]
def get_mitunterzeichner_2(row):
    anzahl_mitunterzeichner = len(row['beteiligte'].split('\n'))
    mitunterzeichner = row['beteiligte'].split('\n')
    if anzahl_mitunterzeichner < 3:
        mitunterzeichner.append('NaN'*(3-anzahl_mitunterzeichner))
    if len(mitunterzeichner) < 3:
        mitunterzeichner.append('NaN'*(3-(len(mitunterzeichner))))
    return mitunterzeichner[2]

In [ ]:
df['mit_1'] = df.apply(get_mitunterzeichner_1, axis=1)
df['mit_2'] = df.apply(get_mitunterzeichner_2, axis=1)
df['mit_1'] = df['mit_1'].str.replace(', Mitunterzeichner\(in\)','')
df['mit_2'] = df['mit_2'].str.replace(', Mitunterzeichner\(in\)','')
df.head()

In [ ]:
#Datensatz der aktuellen Parlamentarier vom Kantonsrat laden.
mit = pd.read_csv('mitglieder_neu.csv', sep=';')
mit['name'] = mit['Name'].str.strip() + ' ' + mit['Vorname'].str.strip() + ' ' + '(' + mit['Partei'] + ')'
mit.head()

In [ ]:
#groupby für Erstunterzeichner
erstunterzeichner = df[['g_erst','g_art']].groupby('g_erst').count().sort_values('g_erst' ,ascending=False).reset_index()
erstunterzeichner

In [ ]:
mitunterzeichner1= df[['mit_1','g_art']].groupby('mit_1').count().sort_values('mit_1' ,ascending=False).reset_index()
mitunterzeichner2= df[['mit_2','g_art']].groupby('mit_2').count().sort_values('mit_2' ,ascending=False).reset_index()
#unterzeichner = pd.concat([erstunterzeichner,mitunterzeichner1,mitunterzeichner2],axis=1).columns
#unterzeichner.columns = ['g_erst','count_erst','g_zweit','zweit_erst','g_erst','count_erst',]

In [ ]:
mitunterzeichner2

In [ ]:
mit_name = pd.DataFrame(mit['name']) #liste der Namen erstellen

In [ ]:
erstunterzeichner['g_erst'] = erstunterzeichner.g_erst.str.replace(', .*', ')')
mitunterzeichner1['mit_1'] = mitunterzeichner1['mit_1'].str.replace(', .*', ')')
mitunterzeichner2['mit_2'] = mitunterzeichner2['mit_2'].str.replace(', .*', ')')

In [ ]:
ranking_erst = pd.merge(erstunterzeichner, mit_name, left_on='g_erst', right_on='name', how='right').sort_values('name', ascending=False)
ranking_zweit = pd.merge(mitunterzeichner1, mit_name, left_on='mit_1', right_on='name', how='right').sort_values('name', ascending=False)
ranking_dritt = pd.merge(mitunterzeichner2, mit_name, left_on='mit_2', right_on='name', how='right').sort_values('name', ascending=False)
ranking_1_2 = pd.merge(ranking_erst, ranking_zweit, on='name')
ranking_all = pd.merge(ranking_1_2, ranking_dritt, on='name')
ranking_all = ranking_all[['name', 'g_art_x', 'g_art_y', 'g_art']]
ranking_all.columns = ['name','erst','mit1','mit2']
ranking_all.fillna(0,inplace=True)
ranking_all['mitunter'] = ranking_all['mit1'] + ranking_all['mit2']
ranking_all = ranking_all[['name','erst','mit1','mit2','mitunter']].sort_values(['erst','mitunter'],ascending=False)
ranking_all = ranking_all[ranking_all.index!=180]
ranking_all
#ranking_all.to_csv('vorstoesse_alle.csv', index=False)


##### Einschub: Kantonsrat-Website scrapen für Politiker Info
Hier relevant: Eintrittsdatum

In [ ]:
# Öffne Chrome Webdriver for Selenium
driver = webdriver.Chrome()

In [ ]:
# gehe auf website
driver.get('https://www.kantonsrat.zh.ch/mitglieder/mitglieder.aspx')

In [ ]:
#wähle feld aus, sende info
nameselector = driver.find_element_by_xpath('//*[@id="tbxName"]')
nameselector.send_keys('Fehr')
prenameselector = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ctl00_ctl00_ctl00_ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_PersonControl_8_tbxVorname"]')
prenameselector.send_keys('Nina')


In [ ]:
#suche knopf, drücke knopf
button = driver.find_element_by_xpath('//*[@id="ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_PersonControl_8_btnSearchPersonen"]') #identify search field
driver.execute_script("arguments[0].scrollIntoView(true)", button) #scroll to the search
button.click()

In [ ]:
#suche link, klicke link
link = driver.find_element_by_xpath('//*[@id="btn1"]')
driver.execute_script("arguments[0].scrollIntoView(true)", link) #scroll to the search
link.click()

In [ ]:
#hole die infos als text
info = driver.find_element_by_class_name('memberDetailInfo').text
info

In [ ]:
#bereite den loop vor (namen putzen, damit es mit der suche klappt)
ranking_all['nachname'] = ranking_all.name.str.extract('(\w*) ')
ranking_all['vorname'] = ranking_all.name.str.extract('(\w*[-]*\w*) \(')
nachname = list(ranking_all['nachname'].values)
vorname = list(ranking_all['vorname'].values)

In [ ]:
# das oben beschrieben in einen Loop packen.
driver = webdriver.Chrome()
infolist = []
for i in range(0,len(nachname)):
    driver.get('https://www.kantonsrat.zh.ch/mitglieder/mitglieder.aspx')
    #select field for year, sende value
    print(i)
    nameselector = driver.find_element_by_xpath('//*[@id="tbxName"]')
    nameselector.send_keys(nachname[i])
    prenameselector = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ctl00_ctl00_ctl00_ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_PersonControl_8_tbxVorname"]')
    prenameselector.send_keys(vorname[i])
    button = driver.find_element_by_xpath('//*[@id="ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_PersonControl_8_btnSearchPersonen"]') #identify search field
    driver.execute_script("arguments[0].scrollIntoView(true)", button) #scroll to the search
    button.click()
    link = driver.find_element_by_xpath('//*[@id="btn1"]')
    driver.execute_script("arguments[0].scrollIntoView(true)", link) #scroll to the search
    link.click()
    info = driver.find_element_by_class_name('memberDetailInfo').text
    infolist.append(info)
ranking_all['info'] = infolist


In [ ]:
#hole das Eintrittsdatum und konvertiere es
ranking_all['eintritt']=ranking_all['info'].str.extract('Eintritt: \n(\d\d.\d\d.\d\d\d\d)')
ranking_all['eintritt'] = ranking_all['eintritt'].str.replace('.', '/')
ranking_all['eintritt'] = pd.to_datetime(ranking_all['eintritt'], format="%d/%m/%Y")
len(ranking_all)

In [ ]:
#Ranking für alle, welche die ganze Legislatur dabei waren.
ranking_volleleg = ranking_all[ranking_all['eintritt']<='2015-05-18']
ranking_volleleg.sort_values(['erst','mitunter'],ascending=False)
ranking_volleleg.head()
#ranking_volleleg.to_csv('ranking_ganzelegislatur.csv', index=False)

In [ ]:
#Ranking für die Newcomer. Erst seit oder während dieser Legislatur dabei.
ranking_neue = ranking_all[ranking_all['eintritt']>='2015-05-18']
ranking_neue.sort_values(['erst','mitunter'],ascending=False)
ranking_neue.head()
#ranking_neue.to_csv('ranking_neue.csv', index=False)